In [1]:
# import os 
# from langchain_teddynote.messages import stream_response
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


from utils.RAGPipeLine import Ragpipeline


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
pipeline = Ragpipeline()

[초기화] vector_store 초기화 완료
[초기화] retriever 초기화 완료
[초기화] RAG chain 초기화 완료


c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


[초기화] RAG title chain 초기화 완료
[초기화] RAG post chain 초기화 완료


In [3]:
retriever = pipeline.init_retriever()
mq_retriever = pipeline.init_multi_query_retriever()
sq_retriever = pipeline.init_self_query_retriever()
pd_retriever  = pipeline.init_parent_document_retriever()
web_retriever = pipeline.init_web_research_retriever()

[초기화] retriever 초기화 완료


In [4]:
query = '인도 통관 및 운송'

docs = retriever.get_relevant_documents(query)

docs

c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='제Ⅲ절\n인도의 통관 절차\n01\n통관 조직 및 절차\n', metadata={'category': '3. 정책 및 무역', 'datetimes': '2024-07-08 16:42:32', 'filename': 'data/PDF_with_contents/3. 정책 및 무역/[정책][관세청][2019.11.07]신남방국 통관·통상환경 및 FTA 활용방안.pdf', 'page_no': 9, 'subcategory': '1. 통관 조직 및 절차'}),
 Document(page_content='인도 전문 식품수입 벤더현황\n다. 통관 및 검역 유의사항\n(1) 인도 통관, 사전 준비가 관건\n▪\n인도의 통관은 수입 통관 및 검역을 위한 사전 준비 절차가 중요함\n▪\n통관에 필요한 서류를 제출하면 서류의 기재 오류나 서류 상호간 불일치를\n수정하기가 매우 어려우며 이로 인해 통관 절차가 2~3주씩 지연되는 경우도\n있어 철저한 서류 작성이 필수적임\n▪\n또한 인도로 수출하는 물품의 경우 포장명세서(P/L)와 상업송장(C/I)을\n컨테이너별로 철저히 작성해야 함. 인도 세관은 한 컨테이너 안에서 박스별로\n화물의 일치 여부를 조사하는 등 실물 검사가 철저하며 서류와 화물 불일치\n시 통관이 불가능함\n▪\n특히 인도는 도로와 항만 등 인프라가 낙후되어 있고 운송수단도 노후하기\n때문에 내륙운송 과정에서 사고가 자주 발생하므로 내륙운송보험에 가입해야\n함\n43\n', metadata={'category': '3. 정책 및 무역', 'datetimes': '2024-07-08 16:41:51', 'filename': 'data/PDF_with_contents/3. 정책 및 무역/[정책][KATI 농식품수출정보][2019]인도 전문 식품수입.pdf', 'page_no': 21, 'subcategory': '다. 통관 및 검역 유의사항'}),
 Document(page_content='5 통관 및 운송\n가 통관제도\n통관 

In [5]:
from utils.prompt import contextualize_q_prompt   
from utils.prompt import qa_prompt2                  # 게시물 생성 프롬프트 
from utils.prompt import title_generator_prompt     # 제목 생성 프롬프트 
from utils.prompt import text_generator_prompt      # 게시물 생성 프롬프트 

### 만든 실험 데이터셋 불러오기

In [6]:
path = './my_data/my_qa.parquet'

# make_qa.py 파일로 만듬 : 질문과 응답 쌍을 만듬 , 어떤 문서가 관련이 있는지 
df = pd.read_parquet(path)

df.head()

,qid,retrieval_gt,query,generation_gt
0,fe24c993-1ea8-468b-97a5-16e3b0ecceff,[[PDF_without_contents\2. 경제 및 시장 분석\[전체 정보][K...,인도에서 채식주의자의 인구 비율이 높은 이유는 무엇인가요?,[인도는 종교적 특성으로 인해 채식주의자의 인구 비율이 높습니다.]
1,a86d92b1-425d-4927-bbef-1b788b277ccc,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코...,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,[상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 ...
2,686973f5-ebcf-4f3a-9c0b-e54e36e4fe82,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인...,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,[인증서의 정지 및 취소는 Schedule II에 명시된 해당 Scheme에 따라 ...
3,2d4c8fd9-597c-4c62-bae0-2655cea70d1a,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인...,불만 사항의 해결 조치는 몇 일 이내에 완료되어야 하나요?,[불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완...
4,9f8d5ec3-56a8-4d33-83e9-75995afea577,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코...,고메카닉(GoMechanic)은 어떤 방식으로 자동차 서비스를 제공하나요?,[고메카닉(GoMechanic)은 집에서 픽업 및 드롭 서비스를 통해 자동차 서비스...


In [7]:
df['retrieval_gt'] = df['retrieval_gt'].apply(lambda x: x[0][0])
df['generation_gt'] = df['generation_gt'].apply(lambda x: x[0])

In [8]:
df.head()

,qid,retrieval_gt,query,generation_gt
0,fe24c993-1ea8-468b-97a5-16e3b0ecceff,PDF_without_contents\2. 경제 및 시장 분석\[전체 정보][KOT...,인도에서 채식주의자의 인구 비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구 비율이 높습니다.
1,a86d92b1-425d-4927-bbef-1b788b277ccc,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코트라...,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...
2,686973f5-ebcf-4f3a-9c0b-e54e36e4fe82,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 해당 Scheme에 따라 이...
3,2d4c8fd9-597c-4c62-bae0-2655cea70d1a,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,불만 사항의 해결 조치는 몇 일 이내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...
4,9f8d5ec3-56a8-4d33-83e9-75995afea577,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코트라...,고메카닉(GoMechanic)은 어떤 방식으로 자동차 서비스를 제공하나요?,고메카닉(GoMechanic)은 집에서 픽업 및 드롭 서비스를 통해 자동차 서비스를...


In [9]:
qa_data = df[['query', 'generation_gt']]
qa_data.columns = ['question', 'ground_truth']

qa_data.head()

,question,ground_truth
0,인도에서 채식주의자의 인구 비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구 비율이 높습니다.
1,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...
2,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 해당 Scheme에 따라 이...
3,불만 사항의 해결 조치는 몇 일 이내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...
4,고메카닉(GoMechanic)은 어떤 방식으로 자동차 서비스를 제공하나요?,고메카닉(GoMechanic)은 집에서 픽업 및 드롭 서비스를 통해 자동차 서비스를...


## 실험을 위한 모델 설정  

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

from utils.config import config

gemma2_llm =  ChatOllama(model="gemma2")
qwen2_llm = ChatOllama(model="qwen2")
our_llm = ChatOpenAI(
            model       = config['llm_predictor']['model_name'],
            temperature = config['llm_predictor']['temperature'],
            # streaming=True,
            # callbacks=[StreamCallback()]
        )

In [11]:
def set_chain(llm, retriever, prompt):
    
    question_answer_chain = create_stuff_documents_chain(llm, prompt)           # 문서 목록을 가져와서 모두 프롬프트로 포맷한 다음 해당 프롬프트를 LLM에 전달합니다.
    
    rag_chat_chain = create_retrieval_chain(retriever, question_answer_chain)  # 사용자 문의를 받아 리트리버로 전달하여 관련 문서를 가져옵니다. 그런 다음 해당 문서(및 원본 입력)는 LLM으로 전달되어 응답을 생성

    
    return rag_chat_chain

In [12]:
rag_chain = set_chain(our_llm, retriever, qa_prompt2)

In [13]:

qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke({'input':x})['answer'])
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)])
# 5분 걸림
qa_data.head()

C:\Users\kim_h\AppData\Local\Temp\ipykernel_33272\597312838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke({'input':x})['answer'])


,question,ground_truth,answer,contexts
0,인도에서 채식주의자의 인구 비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구 비율이 높습니다.,인도에서 채식주의자의 인구 비율이 높은 이유는 종교적 특성과 문화적 영향이 큰 역할...,[이 높게 나타나며 종교적 특성으로 인해 자연제품을 선호하는 경향이 있다\n 채식주...
1,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...,"상표 등록이 완료된 후 기호 ""을""을 사용할 수 있는 조건은 등록된 상표가 소유자의...",[원이 완료되면 기호 을 제품과 함께 사용할 수 있으며 상표 등록이 완료 후에 기호...
2,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 해당 Scheme에 따라 이...,인증서의 정지 및 취소는 일반적으로 다음과 같은 절차에 따라 이루어집니다:\n\n1...,[있다 wwwsafescryptcom wwwnicin wwwidrbtcaorgin ...
3,불만 사항의 해결 조치는 몇 일 이내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...,"불만 사항의 해결 조치는 제공된 자료에 명시된 바에 따르면, ""성격에 맞게 정보를 ...",[채무변제 능력에 대한 이사의 결의에는 다음의 서류가 첨부되어야 한다\ncid0 직...
4,고메카닉(GoMechanic)은 어떤 방식으로 자동차 서비스를 제공하나요?,고메카닉(GoMechanic)은 집에서 픽업 및 드롭 서비스를 통해 자동차 서비스를...,고메카닉(GoMechanic)은 집에서 픽업 및 드롭 서비스를 통해 자동차 서비스를...,[정비 및 시운전 서비스도 출장 방식으로 제공될 수 있다\n고메카닉GoMechani...


In [14]:
from datasets import Dataset 

dataset = Dataset.from_pandas(qa_data)

dataset[0]

{'question': '인도에서 채식주의자의 인구 비율이 높은 이유는 무엇인가요?',
 'ground_truth': '인도는 종교적 특성으로 인해 채식주의자의 인구 비율이 높습니다.',
 'answer': '인도에서 채식주의자의 인구 비율이 높은 이유는 종교적 특성과 문화적 영향이 큰 역할을 합니다. 인도는 다양한 종교가 공존하는 나라이며, 특히 힌두교와 종교적인 이슬람교, 기타 종교들의 영향이 큽니다. \n\n힌두교와 종교적인 이슬람교, 기타 종교들 중 일부는 육식을 금기시하는 경우가 있습니다. 힌두교에서는 소를 신성한 동물로 여기는 경우가 많아 소의 고기를 섭취하지 않는 경우가 많습니다. 이러한 종교적 신념으로 인해 많은 인도인이 채식주의를 따르게 되며, 이는 인도에서 채식주의자의 인구 비율이 높은 이유 중 하나입니다.\n\n또한, 인도의 문화적인 영향도 채식주의를 뒷받침합니다. 인도의 전통적인 음식 문화는 채소와 과일을 중심으로 한 식습관을 가지고 있으며, 이는 채식주의를 선호하는 경향을 높입니다.\n\n따라서, 종교적 신념과 문화적 영향으로 인해 인도에서 채식주의자의 인구 비율이 높게 나타나는 것으로 이해할 수 있습니다.',
 'contexts': ['이 높게 나타나며 종교적 특성으로 인해 자연제품을 선호하는 경향이 있다\n 채식주의\n 인도정부2014에 따르면 인도 내 채식주의자의 비중은 2885이며 나머지는 비 채식주의자로 조사되었다 채식주의자 비중의 지\n역별 편차가 있어 탈렝가나 주의 경우 비 채식주의자의 비중이 987에 이르며 웨스트벵갈 9855 오디샤 9735 케랄라 97\n등 주요 동부와 남부 지역에 위치한 지역의 비 채식주의자 비중이 높게 나타났다',
  '소비 성향\n1 자연친화 제품 소비 경향\n인도는 14억의 인구와 28개 주와 8개 연방 직할지로 구성된 거대 국가이며 공식 언어만 23개에 이르는 등 계층별 지역별 구분이 뚜렷하\n기 때문에 몇 가지 특징으로 인도 전체 소비자의 성향을 재단하기는 매우 어렵다 다만 타 국가와 비교해 

In [15]:
dataset[0]['contexts']

['이 높게 나타나며 종교적 특성으로 인해 자연제품을 선호하는 경향이 있다\n 채식주의\n 인도정부2014에 따르면 인도 내 채식주의자의 비중은 2885이며 나머지는 비 채식주의자로 조사되었다 채식주의자 비중의 지\n역별 편차가 있어 탈렝가나 주의 경우 비 채식주의자의 비중이 987에 이르며 웨스트벵갈 9855 오디샤 9735 케랄라 97\n등 주요 동부와 남부 지역에 위치한 지역의 비 채식주의자 비중이 높게 나타났다',
 '소비 성향\n1 자연친화 제품 소비 경향\n인도는 14억의 인구와 28개 주와 8개 연방 직할지로 구성된 거대 국가이며 공식 언어만 23개에 이르는 등 계층별 지역별 구분이 뚜렷하\n기 때문에 몇 가지 특징으로 인도 전체 소비자의 성향을 재단하기는 매우 어렵다 다만 타 국가와 비교해 봤을 때 채식주의자의 인구비율\n이 높게 나타나며 종교적 특성으로 인해 자연제품을 선호하는 경향이 있다\n 채식주의',
 '적 진입 장벽이 존재한다\n인도의 종교는 힌두교 798 이슬람교 142 기독교개신교천주교 23 시크교\n17 불교 07 자이나교 04 기타 09 비율로 구성되어 있다 각 종교마다 금기시\n되는 사항들이 있는데 인도에서 프랜차이즈 사업을 할 경우 유념해두어야 한다 힌두교 시\n크교 불교 자이나교는 육류 중 닭고기만을 섭취하며 다른 고기류는 먹지 않는다 특히\n소는 힌두교의 신 중 시바신이 타고 다니는 신성한 가축으로 여겼기 때문에 많은 사람들',
 '갖출 수 있는 지역으로의 진출도 고려할 만하다\n이 까닭에 노동집약적 저임금을 기반으로 하는 업종으로 인도로 진출한다는 것은 그다지 바람직\n하지 않다 인적 요소가 기업활동에서 차지하는 비중이 큰 분야일 수록 교육과 훈련 그리고 고용\n유지에 대한 전략이 충분히 검토되어야 한다 그럼에도 불구하고 전반적 인도 인건비 수준은 한국과\n비교하여서는 상대적으로 많이 낮은 편이다 다만 인구가 1억 명에도 미치지 못하는 베트남 등과\n비교하여서 14억 인도의 인건비가 낮지 않은 수준이라는 것은 시사하는 바

In [16]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [17]:
from ragas import evaluate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

langchain_llm =  ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="gemma2")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result

Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\executor.py", line 87, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\base_events.py", line 663, in run_until_complete
    self._check_running()
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\base_events.py", line 622, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\threading.py:1075: RuntimeWarning: coroutine 'Runner._aresults' was never awaited
  self._invoke_excepthook(self)


ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ipykernel\iostream.py:123: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  await self._event_pipe_gc()
